https://benjaminwarner.dev/2023/07/01/attention-mechanism

In [2]:
import torch
import torch.nn as nn
import math

# Single Head Forward 

This Softmax output of $\frac{QK^t}{\sqrt{d_k}}$
​ is how the Attention mechanism weights the strength of the relationship between each pair of tokens. Where higher Softmax values means Attention is placing more importance on these pairs of tokens and lower values are deemed less important.

 Softmax is a function which converts a vector of inputs into a vector of probabilities which are constrained between (0,1)(0,1), sum to one, and reflect the relative scale of each individual input. Or, more formally,
$softmax(x_i)=\frac{e^{x_i}}{\sum_j^K{e^{x_j}}},for i = 1,\dots,K$

In [3]:
class SingleHeadAttention(nn.Module):
    def _init_(self,
               hidden_size: int,
               bias: bool = True):
        super().__init__()
        self.Wqkv = nn.Linear(hidden_size, (hidden_size//4)*3, bias=bias)
        self.Wo = nn.Linear(hidden_size//4, hidden_size, bias=bias)
    
    def forward(self, x:torch.Tensor):
        B, S, C = x.shape
        
        q, k, v = self.Wqkv(x).reshape(B, S, 3, C//4).UNBIND(dim=2)
        
        attn = q @ k.transpose(-2, -1)
        attn = attn / math.sqrt(k.size(-1))
        
        attn = attn.softmax(dim=-1)
        
        x = attn @ v
        
        return self.Wo(x) 

# Multi-Head Self-Attention 

Now that we have our Single Head Self-Attention code understood and working, we can update it to Bidirectional Multi-Head Self-Attention. But first, there is an obvious question which needs to be answered: Why do we want Multi-Head Attention in the first place?

The answer is two parted. First, by projecting the input to multiple randomly initialized heads the Transformer will have multiple representation subspaces for the same input, giving each Transformer layer the ability to simultaneously learn different nuances for the same input tokens.

Second, multiple heads allow the Attention mechanism to jointly attended to multiple tokens at the same time (Although there is a paper which suggests that enough layers of Single Head Attention can perform the same function). Even if a single weighted average is well behaved (A worst case scenario for a Single Headed Attention is the Softmax output only attends to itself or one other token, with all the other tokens contributing a miniscule amount), it still limits the ability to focus on multiple tokens. This ability to attend to multiple tokens at once is especially important as the context window (The context window is the maximum number of tokens in the input sequence that the model was trained or fine-tuned on.) of recent LLMs expands to 4,000, 8,000, 32,000, 60,000, and even 100,000 tokens.

Formally, Multi-Head Attention creates one query $Q_h$​, key $K_h​$, and value $V_h$​ per head $h$, calculates the scaled dot-product Attention per head Attention($Q_h$,$K_h$,$V_h$)A, concatenates all the Attention outputs back into one tensor MultiHead(*Q*,*K*,*V*)MultiHead(*Q*,*K*,*V*), before passing the Multi-Head Attention output through the final linear layer $W_0$:
$$
Q_h = XW^Q_h
$$
$$
K_h = XW^k_h
$$
$$
V_h=XW^V_h
$$
$$
Attetion(Q_h,K_h,V_h) = softmax(\frac{Q_hK^T_h}{\sqrt{d_h}}V_h)
$$
$$
MultiHead(Q,K,V)=concat(Attetion(Q_h,K_h,V_h), for\:all\:h)
$$
$$
Output=MultiHead(Q,K,V)W^O
$$
With Multi-Head Attention formally defined, let’s implement it in code.

In [11]:
class MultiHeadAttetion(nn.Module):
    def __init__(self,
                 hidden_size: int,
                 num_heads: int,
                 attn_drop: float = 0.1,
                 out_drop: float = 0.1,
                 bias: bool = True):
        super().__init__()
        assert hidden_size % num_heads == 0
        self.nh = num_heads
        # linear layer to project queries, keys, values
        self.Wqkv = nn.Linear(hidden_size, hidden_size*3, bias=bias)
        # linear layer to project final output
        self.Wo = nn.Linear(hidden_size, hidden_size, bias=bias)
        # attention dropuout layer to prevent overfitting
        self.attn_drop = nn.Dropout(attn_drop)
        # final output dropout layer to prevent overfitting
        self.out_drop = nn.Dropout(out_drop)
        
    def forward(self, x: torch.Tensor):
        B, S, C = x.shape

        x = self.Wqkv(x).reshape(B, S, 3, self.nh, C//self.nh)
        q, k, v = x.transpose(3, 1).unbind(dim=2)

        attn = q @ k.transpose(-2, -1)
        attn = attn / math.sqrt(k.size(-1))
        # apply softmax to get attention weights (B, NH, S, S)
        attn = attn.softmax(dim=-1)
        # apply dropout to attention weight
        attn = self.attn_drop(attn)
        
        x = attn @ v

        return self.out_drop(self.Wo(x.transpose(1, 2).reshape(B, S, C)))

    

# Bidirectional Attention 

As Bidirectional Attention is supposed to attend to all tokens in the input sequence, the Attention mask primarily exists to support batching different length sequencesAlthough with Nested Tensors on the horizon, the necessity of masking might diminish.. Typically, an encoder or encoder-decoder Transformer will have a pad token, but we don’t want this pad token to interact with any of the sequence tokens. That is where the Attention mask comes in.

Spesso le mie sequenze di entrata hanno lunghezze differenti che vengono riempite con token di padding, ma non voglio che questi token di padding interagiscano con i token della sequenza. Per evitare che ciò accada, posso utilizzare una maschera di attenzione. La maschera di attenzione è una matrice booleana che ha la stessa forma della matrice di attenzione e contiene True dove la matrice di attenzione deve essere calcolata e False dove non deve essere calcolata. In questo caso, la maschera di attenzione è una matrice triangolare superiore con True dove la matrice di attenzione deve essere calcolata e False dove non deve essere calcolata.

In [3]:

class BidirectionalAttention(nn.Module):
    def __init__(self, hidden_size:int, num_heads:int, attn_drop:float=0.1,
                 out_drop:float=0.1, bias:bool=True):
        super().__init__()
        # input dimension must be divisible by num_heads
        assert hidden_size % num_heads == 0
        # number of Attention heads
        self.nh = num_heads

        # linear layer to project queries, keys, values
        self.Wqkv = nn.Linear(hidden_size, hidden_size * 3, bias=bias)

        # attention dropout layer to prevent overfitting
        self.attn_drop = nn.Dropout(attn_drop)

        # linear layer to project final output
        self.Wo = nn.Linear(hidden_size, hidden_size, bias=bias)

        # final output dropout layer to prevent overfitting
        self.out_drop = nn.Dropout(out_drop)

    # boolean `mask` of shape (batch_size, sequence_length)
    # where True is masked and False is unmasked
    def forward(self, x: torch.Tensor, mask: torch.BoolTensor|None = None):
        # batch size, sequence length, input dimension
        B, S, C = x.shape

        # split into queries, keys, & values of shape
        # batch size (B), num_heads (NH), sequence length (S), head size (HS)
        x = self.Wqkv(x).reshape(B, S, 3, self.nh, C//self.nh)
        q, k, v = x.transpose(3, 1).unbind(dim=2)

        # dot product queries and keys for each head
        # (B, NH, S, S) = (B, NH, S, HS) @ (B, NH, HS, S)
        attn = q @ k.transpose(-2, -1)

        # scale by square root of output dimension
        attn = attn / math.sqrt(k.size(-1))

        # reshape and mask attention scores
        if mask is not None:
            attn = attn.masked_fill(mask.view(B, 1, 1, S), float('-inf'))

        # apply softmax to get attention weights
        attn = attn.softmax(dim=-1)

        # apply dropout to attention weight
        attn = self.attn_drop(attn)

        # dot product attention weights with values of shape
        # (B, NH, S, HS) = (B, NH, S, S) @ (B, NH, HS, S)
        x = attn @ v

        # and transpose heads & sequence and reshape back to (B, S, C)
        x = x.transpose(1, 2).reshape(B, S, C)

        # apply final linear layer and dropout to get output (B, S, C)
        return self.out_drop(self.Wo(x))

Come è possibile vedere verso riga 42 se la mask non è null viene ipostato un float ad -inf nella posizione della maschera con masked_fill, quindi quando andremo ad utilizzare la funzione di softmax, queste posizioni avranno un peso di attenzione effettivamente pari a 0, poiché l'esponenziale di -inf è 0.
È importante notare che la maschera deve essere progettata in modo che il suo broadcasting corrisponda alla forma dell'output di attenzione, che è per questo che viene usato mask.view(B, 1, 1, S) per allineare le dimensioni della maschera con quelle dei punteggi di attenzione.

Certo, posso fornirti un esempio di come creare e applicare una maschera in un modulo di attenzione. Supponiamo di avere un batch di sequenze con lunghezze diverse e di voler mascherare i token di padding. Ecco come potresti farlo:

1. **Creazione della Maschera**:
   Supponiamo di avere un batch di sequenze con la seguente lunghezza:
   ```python
   lunghezze_sequenze = [3, 5, 4]  # Esempio di lunghezze delle sequenze nel batch
   max_len = max(lunghezze_sequenze)  # La lunghezza massima della sequenza nel batch
   ```

   Creiamo una maschera che abbia `False` per i token reali e `True` per i token di padding:
   ```python
   mask = torch.ones((len(lunghezze_sequenze), max_len), dtype=torch.bool)
   for i, lunghezza in enumerate(lunghezze_sequenze):
       mask[i, :lunghezza] = False
   ```

   Ora `mask` è un tensore che assomiglia a questo (per il nostro esempio di lunghezze):
   ```python
   tensor([[False, False, False,  True,  True],
           [False, False, False, False, False],
           [False, False, False, False,  True]])
   ```

2. **Applicazione della Maschera nel Modulo di Attenzione**:
   Quando passi la maschera al tuo modulo di attenzione, la maschera viene utilizzata per impostare i punteggi di attenzione a `-inf` per i token di padding, come mostrato nel tuo codice:
   ```python
   if mask is not None:
       attn = attn.masked_fill(mask.unsqueeze(1).unsqueeze(2), float('-inf'))
   ```

   La funzione `unsqueeze` è usata per aggiungere dimensioni in modo che la maschera si allinei correttamente con la forma dei punteggi di attenzione `(B, NH, S, S)`.

3. **Esempio Completo**:
   Ecco un esempio completo che include la creazione di un tensore di input fittizio, la creazione di una maschera e l'applicazione della maschera in un passaggio in avanti attraverso il modulo di attenzione:
   ```python
   import torch
   import torch.nn as nn
   import math

   # Supponiamo che il nostro modulo di attenzione sia definito come nel tuo esempio
   class BidirectionalAttention(nn.Module):
       # ... (il resto del codice del modulo)

       def forward(self, x: torch.Tensor, mask: torch.BoolTensor|None = None):
           # ... (il resto del codice del passaggio in avanti)
           if mask is not None:
               attn = attn.masked_fill(mask.unsqueeze(1).unsqueeze(2), float('-inf'))
           # ... (il resto del codice del passaggio in avanti)

   # Creazione di un tensore di input fittizio (B, S, C)
   B, S, C = 3, 5, 8  # Batch size, Sequence length, Channels
   x = torch.rand(B, S, C)

   # Creazione della maschera
   lunghezze_sequenze = [3, 5, 4]
   mask = torch.ones((B, S), dtype=torch.bool)
   for i, lunghezza in enumerate(lunghezze_sequenze):
       mask[i, :lunghezza] = False

   # Creazione di un'istanza del modulo di attenzione e passaggio in avanti
   attn_module = BidirectionalAttention(hidden_size=C, num_heads=2)
   output = attn_module(x, mask=mask)

   # Stampa l'output
   print(output)
   ```

In questo esempio, abbiamo creato un tensore di input casuale e una maschera basata sulle lunghezze delle sequenze. Abbiamo poi passato entrambi attraverso il modulo di attenzione, che applica la maschera durante il calcolo dei punteggi di attenzione.

Nel contesto dei modelli di linguaggio (Language Models, LLM), il padding si riferisce all'aggiunta di token non significativi a una sequenza di testo per portarla a una lunghezza fissa. Questo è spesso necessario perché molti modelli di machine learning richiedono input di dimensioni uniformi. Ci sono due tipi principali di padding che possono essere utilizzati: il "left padding" (o pre-padding) e il "right padding" (o post-padding).

Right Padding (Post-Padding):

Il right padding aggiunge token di padding alla fine di una sequenza.
È il tipo di padding più comune nei modelli di linguaggio perché i modelli sono spesso progettati per elaborare l'input da sinistra a destra. Aggiungendo il padding alla fine, l'informazione iniziale (che è spesso la più rilevante per la comprensione del contesto) viene mantenuta all'inizio della sequenza, dove il modello inizia a elaborarla.
Nei modelli di sequenza come RNN, LSTM o GRU, il right padding è preferibile perché l'informazione rilevante viene elaborata per prima e il padding alla fine ha meno probabilità di influenzare l'output del modello.
Left Padding (Pre-Padding):

Il left padding aggiunge token di padding all'inizio di una sequenza.
Questo tipo di padding può essere utile in alcuni casi specifici, come quando si utilizzano modelli che elaborano l'input da destra a sinistra o quando si utilizzano meccanismi di attenzione che possono facilmente ignorare il padding all'inizio della sequenza.
In alcuni modelli Transformer, come BERT, il left padding può essere utilizzato perché il modello è in grado di elaborare l'intera sequenza contemporaneamente (attenzione globale) e non è influenzato dall'ordine in cui i token vengono presentati.
La scelta tra right e left padding dipende dalla struttura e dall'architettura del modello di linguaggio che si sta utilizzando, nonché dalla natura del compito di elaborazione del linguaggio naturale. In generale, il right padding è più comune, ma è importante considerare le specifiche del modello e del compito per decidere quale approccio di padding sia più appropriato.

# Causal Self-Attention 

For Causal Attention, we need to ensure that current tokens can only attend to past tokens, and not future tokens in the sequence. We can accomplish this through masking.

We will use an upper triangular matrix for the Causal Attention mask to ensure the current token can only attend to past tokens no matter where the current token is in the sequence. Figure 7 illustrates how the upper triangular matrix is applied on a per-token level, where the diagonal, (1,1)(1,1), (2,2)(2,2), etc, is the current token in the sequence. Green shaded tokens, both the current token and tokens to the left of the current token, are unmasked and can be attended too, while grey shaded tokens to the right of the current token are masked and cannot used in the Attention mechanism.

We’ll create a permanent causal_mask of shape [context_size, context_size] in our CausalAttention initialization method, where context_size is the maximum context length of our Transformer. To match our padding Attention maskWhere True is masked and False is unmasked. we will create a matrix of boolean ones. Then we use triu to convert our boolean matrix of True values into an upper triangular matrix, with the upper triangle masked (True) and lower triangle unmasked (False). Because we want the diagonal of the matrix to be unmasked, we shift the triu diagonal one to the upper-right using diagonal=1.

Then we reshape the input to be broadcastable across the dimensions of $QK^T$, which is B, NH, S, S, and assign it to a PyTorch bufferThis insures the values are not considered parameters, and thus will not be modified by an optimizer..

Certo, ti spiegherò il concetto di Causal Attention e come viene implementato attraverso il mascheramento utilizzando una matrice triangolare superiore.

**Causal Attention**:
La Causal Attention è un tipo di attenzione utilizzato nei modelli di generazione di testo come GPT (Generative Pretrained Transformer). L'idea è che quando generi un token in una posizione specifica, dovresti essere in grado di considerare solo i token precedenti (passati), non quelli futuri. Questo perché, durante la generazione di testo, i token futuri non sono ancora stati generati e quindi non dovrebbero influenzare la generazione del token corrente.

**Mascheramento con Matrice Triangolare Superiore**:
Per implementare la Causal Attention, si utilizza una matrice triangolare superiore come maschera. Questa matrice ha valori di `True` (o un valore equivalente che indica un mascheramento, come `-inf` dopo l'applicazione di `softmax`) nella parte superiore triangolare e `False` nella parte inferiore triangolare e lungo la diagonale.

**Esempio di Matrice Triangolare Superiore**:
```
[[False, True,  True,  True],
 [False, False, True,  True],
 [False, False, False, True],
 [False, False, False, False]]
```
In questa matrice, `False` indica che il token può essere considerato nell'attenzione (non mascherato), mentre `True` indica che il token non deve essere considerato (mascherato).

**Implementazione**:
Quando si inizializza il modulo di Causal Attention, si crea una maschera permanente di forma `[context_size, context_size]`, dove `context_size` è la lunghezza massima del contesto che il Transformer può considerare.

Per creare questa maschera, si inizia con una matrice di booleani con tutti i valori impostati su `True`. Poi si utilizza la funzione `torch.triu` (triangolare superiore) per convertire questa matrice in una matrice triangolare superiore. Poiché si desidera che la diagonale sia non mascherata (permettendo a ogni token di "vedere" se stesso), si sposta la diagonale della matrice triangolare superiore di uno verso l'alto e verso destra impostando `diagonal=1`.

**Codice di Esempio**:
```python
import torch

context_size = 4  # Esempio di lunghezza massima del contesto
causal_mask = torch.triu(torch.ones(context_size, context_size), diagonal=1).bool()
```
Questo produrrà una maschera come quella mostrata nell'esempio di matrice sopra.

**Buffer in PyTorch**:
Infine, si assegna questa maschera a un buffer in PyTorch. Un buffer in PyTorch è un tipo di tensore che non è considerato un parametro del modello, il che significa che non verrà aggiornato durante il processo di ottimizzazione (addestramento). Questo è importante perché la maschera causale è una parte fissa dell'architettura del modello e non deve cambiare durante l'addestramento.

**Applicazione della Maschera**:
Durante il calcolo dell'attenzione, la maschera viene applicata ai punteggi di attenzione (solitamente dopo aver calcolato il prodotto punto tra query e key, ma prima di applicare la funzione `softmax`) in modo che i token futuri siano effettivamente ignorati.

**Riassumendo**, la Causal Attention assicura che ogni token possa essere influenzato solo dai token precedenti nella sequenza, e questo viene realizzato attraverso un mascheramento strategico con una matrice triangolare superiore. Questo tipo di attenzione è essenziale per i modelli generativi che producono sequenze token per token in un ordine sequenziale.

In [ ]:
class CausalAttention(nn.Module):
    def __init__(self, hidden_size:int, num_heads:int, context_size:int,
                 attn_drop:float=0.1, out_drop:float=0.1, bias:bool=True):
        super().__init__()
        # input dimension must be divisible by num_heads
        assert hidden_size % num_heads == 0
        # number of Attention heads
        self.nh = num_heads

        # linear layer to project queries, keys, values
        self.Wqkv = nn.Linear(hidden_size, hidden_size * 3, bias=bias)

        # attention dropout layer to prevent overfitting
        self.attn_drop = nn.Dropout(attn_drop)

        # linear layer to project final output
        self.Wo = nn.Linear(hidden_size, hidden_size, bias=bias)

        # final output dropout layer to prevent overfitting
        self.out_drop = nn.Dropout(out_drop)

        # causal mask to ensure that Attention is not applied to future tokens where
        # context_size is the maximum sequence length of the transformer
        self.register_buffer('causal_mask',
            torch.triu(torch.ones([context_size, context_size], dtype=torch.bool), diagonal=1)
                .view(1, 1, context_size, context_size), persistent=False
        )

    # boolean `mask` of shape (batch_size, sequence_length)
    # where True is masked and False is unmasked
    def forward(self, x: Tensor, mask: BoolTensor|None = None):
        # batch size, sequence length, input dimension
        B, S, C = x.shape

        # split into queries, keys, & values of shape
        # batch size (B), num_heads (NH), sequence length (S), head size (HS)
        x = self.Wqkv(x).reshape(B, S, 3, self.nh, C//self.nh)
        q, k, v = x.transpose(3, 1).unbind(dim=2)

        # dot product queries and keys for each head
        # (B, NH, S, S) = (B, NH, S, HS) @ (B, NH, HS, S)
        attn = q @ k.transpose(-2, -1)

        # scale by square root of output dimension
        attn = attn / math.sqrt(k.size(-1))

        # apply input and causal mask
        combined_mask = self.causal_mask[:, :, :S, :S]
        if mask is not None:
            combined_mask += mask.view(B, 1, 1, S)
        attn = attn.masked_fill(combined_mask, float('-inf'))

        # apply softmax to get attention weights
        attn = attn.softmax(dim=-1)

        # apply dropout to attention weight
        attn = self.attn_drop(attn)

        # dot product attention weights with values of shape
        # (B, NH, S, HS) = (B, NH, S, S) @ (B, NH, HS, S)
        x = attn @ v

        # and transpose heads & sequence and reshape back to (B, S, C)
        x = x.transpose(1, 2).reshape(B, S, C)

        # apply final linear layer and dropout to get output (B, S, C)
        return self.out_drop(self.Wo(x))

# Cross Attention 

In [ ]:
class CausalCrossAttention(nn.Module):
    def __init__(self,
        hidden_size: int,
        num_heads: int,
        context_size: int,
        attn_drop: float = 0.1,
        out_drop: float = 0.1,
        bias: bool = True,
    ):
        super().__init__()
        # input dimension must be divisible by num_heads
        assert hidden_size % num_heads == 0
        # number of Attention heads
        self.nh = num_heads

        # linear layer to project queries from decoder input
        self.Wq = nn.Linear(hidden_size, hidden_size, bias=bias)

        # linear layer to project keys and values from encoder output
        self.Wkv = nn.Linear(hidden_size, hidden_size * 2, bias=bias)

        # attention dropout layer to prevent overfitting
        self.attn_drop = nn.Dropout(attn_drop)

        # linear layer to project final output
        self.Wo = nn.Linear(hidden_size, hidden_size, bias=bias)

        # final output dropout layer to prevent overfitting
        self.out_drop = nn.Dropout(out_drop)

        # causal mask to ensure that Attention is not applied to future tokens where
        # context_size is the maximum sequence length of the transformer
        self.register_buffer('causal_mask',
            torch.triu(torch.ones([context_size, context_size], dtype=torch.bool), diagonal=1)
                .view(1, 1, context_size, context_size), persistent=False
        )


    # boolean `mask` of shape (batch_size, sequence_length)
    # where True is masked and False is unmasked
    def forward(self, x: Tensor, y: Tensor, mask: BoolTensor|None = None):
        # batch size, sequence length, input dimension
        B, S, C = x.shape

        # split into queries of shape (B, NH, S, HS) from decoder input
        q = self.Wq(x).reshape(B, S, self.nh, C//self.nh).transpose(1, 2)

        # split into keys and values of shape (B, NH, S, HS) from encoder output
        y = self.Wkv(y).reshape(B, S, 2, self.nh, C//self.nh)
        k, v = y.transpose(3, 1).unbind(dim=2)

        # dot product queries and keys for each head
        # (B, NH, S, S) = (B, NH, S, HS) @ (B, NH, HS, S)
        attn = q @ k.transpose(-2, -1)

        # scale by square root of output dimension
        attn = attn / math.sqrt(k.size(-1))

        # apply input and causal mask
        combined_mask = self.causal_mask[:, :, :S, :S]
        if mask is not None:
            combined_mask += mask.view(B, 1, 1, S)
        attn = attn.masked_fill(combined_mask, float('-inf'))

        # apply softmax to get attention weights
        attn = attn.softmax(dim=-1)

        # apply dropout to attention weight
        attn = self.attn_drop(attn)

        # dot product attention weights with values of shape
        # (B,NH,S,S) @ (B,NH,S,HS) -> (B,NH,S,HS)
        x = attn @ v

        # and transpose heads & sequence and reshape back to (B,S,C)
        x = x.transpose(1, 2).reshape(B, S, C)

        # apply final linear layer and dropout to get output (B,S,C)
        return self.out_drop(self.Wo(x))

# Conclusion

In this post, I have shown you how to implement all three main flavors of Attention in PyTorch: Bidirectional, Causal, and Cross Attention. You should now be able to write your own version of Attention and understand any model-specific Attention implementations.

There still are a few more items we need to create before we have a fully working Transformer: the feed-forward network, positional encoding, and text embedding layers, to name a few. In the next post in this series, I will show you how to create all of these in PyTorch and build the rest of the Transformer.
